This example reads snapshot data and creates a slowly changing dimension from it using hashing and window functions.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SvnLocalSpark") \
    .config("spark.jars", "../scala-udaf/target/scala-2.13/svn-local-spark_2.13-0.1.0-SNAPSHOT.jar") \
    .master("local")\
    .getOrCreate()

print(f"spark {spark.version} {spark.sparkContext.uiWebUrl}")

spark 3.5.4 http://DESKTOP-4GOMK6M:4040


In [2]:
spark.sql("CREATE SCHEMA IF NOT EXISTS landing")
# create external table
spark.catalog.getDatabase("landing")
spark.catalog.createTable(
    tableName = "landing.commercial_properties",
    source = "csv",
    description = "property values",
    header="true", delimiter=",", path="../../../resources/sourcedata/commercial_property_snapshots_100_M39.csv", inferSchema="true")

raw = spark.table("landing.commercial_properties")

In [3]:
raw.where("property_id='P012'").orderBy("date").limit(20).show()

+----------+-----------+---------------+-------------+----------+--------+--------+--------------+------------+
|      date|property_id|         street|street_number|      city|zip_code|category|property_value|energy_label|
+----------+-----------+---------------+-------------+----------+--------+--------+--------------+------------+
|2022-01-01|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     187462.36|           C|
|2022-01-02|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     187462.36|           C|
|2022-01-03|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     187462.36|           C|
|2022-01-04|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     187462.36|           C|
|2022-01-05|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     191214.34|           C|
|2022-01-06|       P012|Carolina Avenue|          847|Wilmington|   28777|  Office|     191214.34|      

In [4]:
%load_ext sparksql_magic

In [5]:
java_catalyst_udf = spark.sparkContext._jvm.spark.udaf.SvnFunctionRegistration.registerFunctions(spark._jsparkSession)

In [6]:
%%sparksql
SELECT `date`, city,property_value, energy_label
, lead_unequal(`date`,property_value) OVER(PARTITION BY property_id ORDER BY `date` ASC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) as v1
, lead_unequal(`date`,property_value) OVER(PARTITION BY property_id ORDER BY `date` ASC) as v2
, lead_unequal(`date`,energy_label) OVER(PARTITION BY property_id ORDER BY `date` ASC ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) as v5
FROM landing.commercial_properties
WHERE property_id='P015'
AND `date` IN ('2022-01-01', '2022-01-02', '2022-01-05'
    ,'2022-01-10','2022-01-11','2022-01-12','2022-01-13'
    ,'2024-12-21','2024-12-22','2024-12-23','2024-12-24')
ORDER BY `date`

date,city,property_value,energy_label,v1,v2,v5
2022-01-01,Concord,211106.16,C,2022-01-11,null,2024-12-21
2022-01-02,Concord,211106.16,C,2022-01-11,null,2024-12-21
2022-01-05,Concord,211106.16,C,2022-01-11,null,2024-12-21
2022-01-10,Concord,211106.16,C,2022-01-11,null,2024-12-21
2022-01-11,Concord,210828.69,C,2022-01-12,2022-01-11,2024-12-21
2022-01-12,Concord,208946.05,C,2024-12-21,2022-01-11,2024-12-21
2022-01-13,Concord,208946.05,C,2024-12-21,2022-01-11,2024-12-21
2024-12-21,Concord,388730.05,A,2024-12-23,2022-01-11,2024-12-23
2024-12-22,Concord,388730.05,A,2024-12-23,2022-01-11,2024-12-23
2024-12-23,Concord,401243.99,E,null,2022-01-11,null


In [7]:
%%sparksql
SELECT property_id
    ,`date` AS valid_from
    ,COALESCE(LEAD(`date`) OVER(PARTITION BY property_id ORDER BY `date`), to_date('9999-12-31')) AS valid_to    
    ,street,street_number,city,zip_code,category,property_value,energy_label
FROM (
    SELECT *
        ,LAG(hash) OVER(PARTITION BY property_id ORDER BY `date`) as prevHash
    FROM (
        SELECT *
            ,SHA2(CONCAT(street,street_number,city,zip_code,category,property_value,energy_label),512) AS hash
        FROM landing.commercial_properties) AS t1
    ) as t2
WHERE (t2.hash<>t2.prevHash OR t2.prevHash IS NULL)
AND t2.property_id='P012'
ORDER BY t2.`date`

only showing top 20 row(s)


property_id,valid_from,valid_to,street,street_number,city,zip_code,category,property_value,energy_label
P012,2022-01-01,2022-01-05,Carolina Avenue,847,Wilmington,28777,Office,187462.36,C
P012,2022-01-05,2022-01-18,Carolina Avenue,847,Wilmington,28777,Office,191214.34,C
P012,2022-01-18,2022-02-19,Carolina Avenue,847,Wilmington,28777,Office,196199.71,C
P012,2022-02-19,2022-03-13,Carolina Avenue,847,Wilmington,28777,Office,200512.03,C
P012,2022-03-13,2022-03-29,Carolina Avenue,847,Wilmington,28777,Office,199764.19,C
P012,2022-03-29,2022-04-26,Carolina Avenue,847,Wilmington,28777,Office,205549.8,C
P012,2022-04-26,2022-04-29,Carolina Avenue,847,Wilmington,28777,Office,202322.08,C
P012,2022-04-29,2022-06-02,Carolina Avenue,847,Wilmington,28777,Office,199776.09,C
P012,2022-06-02,2022-06-07,Carolina Avenue,847,Wilmington,28777,Office,200637.3,C
P012,2022-06-07,2022-06-19,Carolina Avenue,847,Wilmington,28777,Office,201827.14,C


In [8]:
java_udf1 = spark.sparkContext._jvm.spark.udaf.LeadUnequalDateString.register(spark._jsparkSession)

In [9]:
%%sparksql
SELECT property_id
    ,`date` AS valid_from
    ,valid_to    
    ,street,street_number,city,zip_code,category,property_value,energy_label
FROM (
    SELECT *
        ,LAG(hash) OVER(PARTITION BY property_id ORDER BY `date`) as prevHash
        ,COALESCE(LEAD_UNEQUAL_DATE_STRING(`date`, hash) OVER(PARTITION BY property_id ORDER BY `date` ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING), to_date('9999-12-31')) as valid_to
    FROM (
        SELECT *
            ,SHA2(CONCAT(street,street_number,city,zip_code,category,property_value,energy_label),512) AS hash
        FROM landing.commercial_properties) AS t1
    ) as t2
WHERE (t2.hash<>t2.prevHash OR t2.prevHash IS NULL)
AND t2.property_id='P012'
ORDER BY t2.`date`

only showing top 20 row(s)


property_id,valid_from,valid_to,street,street_number,city,zip_code,category,property_value,energy_label
P012,2022-01-01,2022-01-05,Carolina Avenue,847,Wilmington,28777,Office,187462.36,C
P012,2022-01-05,2022-01-18,Carolina Avenue,847,Wilmington,28777,Office,191214.34,C
P012,2022-01-18,2022-02-19,Carolina Avenue,847,Wilmington,28777,Office,196199.71,C
P012,2022-02-19,2022-03-13,Carolina Avenue,847,Wilmington,28777,Office,200512.03,C
P012,2022-03-13,2022-03-29,Carolina Avenue,847,Wilmington,28777,Office,199764.19,C
P012,2022-03-29,2022-04-26,Carolina Avenue,847,Wilmington,28777,Office,205549.8,C
P012,2022-04-26,2022-04-29,Carolina Avenue,847,Wilmington,28777,Office,202322.08,C
P012,2022-04-29,2022-06-02,Carolina Avenue,847,Wilmington,28777,Office,199776.09,C
P012,2022-06-02,2022-06-07,Carolina Avenue,847,Wilmington,28777,Office,200637.3,C
P012,2022-06-07,2022-06-19,Carolina Avenue,847,Wilmington,28777,Office,201827.14,C
